In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
!pip install sklearn --upgrade
!pip install joblib

Requirement already up-to-date: sklearn in c:\users\pengu\anaconda3\lib\site-packages (0.0)


In [2]:
steamdf = pd.read_csv('mergedf.csv')
steamdf.head()

,App Id,Name,Developers,Publishers,Metascores,Genres,Tags,Price,clean_genres,clean_tags,min owners,max owners,Hit
0,10,Counter-Strike,Valve,Valve,88%,['Action'],"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",$9.99,['Action'],"['Action','FPS','Multiplayer','Shooter','Class...",10000000,20000000,1.0
1,30,Day of Defeat,Valve,Valve,79%,['Action'],"['FPS', 'World War II', 'Multiplayer', 'Shoote...",$4.99,['Action'],"['FPS','World_War_II','Multiplayer','Shooter',...",5000000,10000000,1.0
2,70,Half-Life,Valve,Valve,96%,['Action'],"['FPS', 'Classic', 'Action', 'Sci-fi', 'Single...",$9.99,['Action'],"['FPS','Classic','Action','Sci-fi','Singleplay...",5000000,10000000,1.0
3,80,Counter-Strike: Condition Zero,Valve,Valve,65%,['Action'],"['Action', 'FPS', 'Shooter', 'Multiplayer', 'F...",$9.99,['Action'],"['Action','FPS','Shooter','Multiplayer','First...",5000000,10000000,1.0
4,100,Counter-Strike: Condition Zero,Valve,Valve,65%,['Action'],"['Action', 'FPS', 'Shooter', 'Multiplayer', 'F...",$9.99,['Action'],"['Action','FPS','Shooter','Multiplayer','First...",10000000,20000000,1.0


# Data Cleaning and Preparation

In [3]:
# Drop the null rows
steamdf = steamdf.dropna()

#Change format of metascore and price so just integer
steamdf['Metascores'] = steamdf['Metascores'].str.rstrip('%').astype('float') / 100.0
steamdf['Price'] = steamdf['Price'].str.replace('$', '').astype('float')

#Drop columns we dont need/want
#Removing owners since we based hit off those
cleandf = steamdf.drop(['Name', 'Publishers', 'min owners', 'max owners', 'clean_genres', 'clean_tags'], axis=1)
cleandf.head()

,App Id,Developers,Metascores,Genres,Tags,Price,Hit
0,10,Valve,0.88,['Action'],"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",9.99,1.0
1,30,Valve,0.79,['Action'],"['FPS', 'World War II', 'Multiplayer', 'Shoote...",4.99,1.0
2,70,Valve,0.96,['Action'],"['FPS', 'Classic', 'Action', 'Sci-fi', 'Single...",9.99,1.0
3,80,Valve,0.65,['Action'],"['Action', 'FPS', 'Shooter', 'Multiplayer', 'F...",9.99,1.0
4,100,Valve,0.65,['Action'],"['Action', 'FPS', 'Shooter', 'Multiplayer', 'F...",9.99,1.0


In [4]:
# Need to split Genres & Tags
cleandf['Genres']=cleandf["Genres"].str.replace(' ','_').str.replace(',_', ',')
cleandf['Tags']=cleandf["Tags"].str.replace(' ','_').str.replace(',_', ',')
cleandf['Developers']=cleandf["Developers"].str.replace(' ','_').str.replace(',_', ',')
cleandf

,App Id,Developers,Metascores,Genres,Tags,Price,Hit
0,10,Valve,0.88,['Action'],"['Action','FPS','Multiplayer','Shooter','Class...",9.99,1.0
1,30,Valve,0.79,['Action'],"['FPS','World_War_II','Multiplayer','Shooter',...",4.99,1.0
2,70,Valve,0.96,['Action'],"['FPS','Classic','Action','Sci-fi','Singleplay...",9.99,1.0
3,80,Valve,0.65,['Action'],"['Action','FPS','Shooter','Multiplayer','First...",9.99,1.0
4,100,Valve,0.65,['Action'],"['Action','FPS','Shooter','Multiplayer','First...",9.99,1.0
...,...,...,...,...,...,...,...
4000,1286830,BioWare,0.85,"['Free_to_Play','Massively_Multiplayer','RPG']","['Free_to_Play','MMORPG','Multiplayer','RPG','...",0.00,1.0
4001,1311510,Mattis_Folkestad,0.86,"['Adventure','Indie']","['Adventure','Indie','Story_Rich','Music','Exp...",12.99,0.0
4002,1328660,Stellar_Entertainment_Limited,0.74,"['Action','Racing']","['Combat_Racing','Racing','Open_World','Drivin...",29.99,0.0
4003,1342620,Different_Tales,0.78,"['Adventure','Indie','RPG']","['Werewolves','Visual_Novel','RPG','Psychologi...",14.99,1.0


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
#genres split
genres=cleandf['Genres'].dropna().str.replace("'", '').str.replace('[', '').str.replace(']', '')
count_vec=CountVectorizer()
encoded=count_vec.fit_transform(genres)
count_vec.vocabulary_

encoded_genres= pd.DataFrame(encoded.toarray(), columns=count_vec.vocabulary_)
encoded_genres

,action,free_to_play,strategy,adventure,indie,rpg,casual,simulation,racing,massively_multiplayer,...,gore,early_access,sexual_content,animation_,_modeling,education,utilities,video_production,movie,documentary
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3939,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3940,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3941,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3942,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [6]:
#Tag split
tags=cleandf['Tags'].dropna().str.replace("'", '').str.replace('[', '').str.replace(']', '')
count_vec=CountVectorizer()
encoded_tags=count_vec.fit_transform(tags)
count_vec.vocabulary_

encoded_tags= pd.DataFrame(encoded_tags.toarray(), columns=count_vec.vocabulary_)
encoded_tags

,action,fps,multiplayer,shooter,classic,team,based,first,person,competitive,...,bmx,faith,idler,foreign,social_deduction,utilities,video_production,game_development,trivia,auto_battler
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3942,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
#Developer split
developers=cleandf['Developers'].dropna().str.replace("'", '').str.replace('[', '').str.replace(']', '')
count_vec=CountVectorizer()
encoded_dev=count_vec.fit_transform(developers)
count_vec.vocabulary_

encoded_dev= pd.DataFrame(encoded_dev.toarray(), columns=count_vec.vocabulary_)
encoded_dev

,valve,gearbox_software,mark_healey,tripwire_interactive,ritual_entertainment,introversion_software,sonalysts,malfador_machinations,strategy_first,arkane_studios,...,robotpumpkin_games,filipe_f,_thomaz,draw_me_a_pixel,dlala_studios,lizard_hazard_games,popcap,criterion_software,stellar_entertainment_limited,different_tales
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3942,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#merge split tags and genres into one df
merge_df = encoded_tags.merge(encoded_genres,left_index=True, right_index=True)
merge2_df = merge_df.merge(encoded_dev,left_index=True, right_index=True)
model_df = merge2_df.merge(cleandf,left_index=True, right_index=True)
model_df = model_df.drop(['Genres', 'Tags', 'Developers'], axis=1)
model_df


,action_x,fps,multiplayer,shooter,classic,team,based,first,person,competitive,...,dlala_studios,lizard_hazard_games,popcap,criterion_software,stellar_entertainment_limited,different_tales,App Id,Metascores,Price,Hit
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,0.88,9.99,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,30,0.79,4.99,1.0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,70,0.96,9.99,1.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,80,0.65,9.99,1.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,100,0.65,9.99,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1019450,0.51,17.99,0.0
3940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1019590,0.69,9.99,0.0
3941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1020340,0.77,29.99,0.0
3942,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1161880,0.72,17.99,0.0


In [32]:
#save csv
model_df.to_csv('modeldf.csv', index = False, header=True)

In [17]:
# App Id | Valve | Developer 2 | Metascores | Action | Thriller | Action | FPS | Hit
# Categorical - 'Developers', 'Genres', 'Tags' (Binary after OH Encoding)
# Ordinal - Metascores, Price

# Select your features (columns)

In [18]:
# Set features. This will also be used as your x values.
selected_features = model_df.drop("Hit", axis=1)

# Create a Train Test Split

In [19]:
y = model_df["Hit"]
print(selected_features.shape, y.shape)

(3883, 2606) (3883,)


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=1)
X_train.head()

,action_x,fps,multiplayer,shooter,classic,team,based,first,person,competitive,...,draw_me_a_pixel,dlala_studios,lizard_hazard_games,popcap,criterion_software,stellar_entertainment_limited,different_tales,App Id,Metascores,Price
3640,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,760590,0.77,0.00
996,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,102500,0.81,0.00
673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,39650,0.61,9.99
1898,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,272510,0.61,4.79
2274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,324260,0.59,14.99


In [21]:
#Preprocessing

# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [25]:
#Train the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=2606))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='sigmoid'))

In [26]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               260700    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 271,002
Trainable params: 271,002
Non-trainable params: 0
_________________________________________________________________


In [27]:
# model.summary()
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Epoch 1/60
91/91 - 0s - loss: 0.6855 - accuracy: 0.5474
Epoch 2/60
91/91 - 0s - loss: 0.6173 - accuracy: 0.6556
Epoch 3/60
91/91 - 0s - loss: 0.4845 - accuracy: 0.7761
Epoch 4/60
91/91 - 0s - loss: 0.3323 - accuracy: 0.8647
Epoch 5/60
91/91 - 0s - loss: 0.2226 - accuracy: 0.9121
Epoch 6/60
91/91 - 0s - loss: 0.1548 - accuracy: 0.9396
Epoch 7/60
91/91 - 0s - loss: 0.1227 - accuracy: 0.9505
Epoch 8/60
91/91 - 0s - loss: 0.0981 - accuracy: 0.9598
Epoch 9/60
91/91 - 0s - loss: 0.0800 - accuracy: 0.9677
Epoch 10/60
91/91 - 0s - loss: 0.0705 - accuracy: 0.9698
Epoch 11/60
91/91 - 0s - loss: 0.0646 - accuracy: 0.9722
Epoch 12/60
91/91 - 0s - loss: 0.0621 - accuracy: 0.9760
Epoch 13/60
91/91 - 0s - loss: 0.0498 - accuracy: 0.9784
Epoch 14/60
91/91 - 0s - loss: 0.0486 - accuracy: 0.9794
Epoch 15/60
91/91 - 0s - loss: 0.0417 - accuracy: 0.9825
Epoch 16/60
91/91 - 0s - loss: 0.0451 - accuracy: 0.9791
Epoch 17/60
91/91 - 0s - loss: 0.0490 - accuracy: 0.9794
Epoch 18/60
91/91 - 0s - loss: 0.0436 - 

In [28]:
# Quantify the model
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

31/31 - 0s - loss: 2.3905 - accuracy: 0.5757
Normal Neural Network - Loss: 2.3904738426208496, Accuracy: 0.5756951570510864


In [29]:
#Predictions
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Predicted classes: [0. 1. 1. 1. 1.]
Actual Labels: [0.0, 0.0, 0.0, 1.0, 0.0]


In [30]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train_scaled, y_train_categorical)
rf.feature_importances_

array([0.00091038, 0.0012393 , 0.00515573, ..., 0.06754018, 0.06115323,
       0.06059071])

In [31]:
list(zip(model_df.columns[:-1], rf.feature_importances_))

[('action_x', 0.0009103770787731617),
 ('fps', 0.0012392958210877345),
 ('multiplayer', 0.0051557283275789866),
 ('shooter', 0.0004305596535703639),
 ('classic', 0.0005375157040691367),
 ('team', 0.001501627904628815),
 ('based', 0.00042152624379123224),
 ('first', 0.0012887222130035225),
 ('person', 0.0007456607676877692),
 ('competitive', 0.001722707428282565),
 ('tactical', 0.0016182813550363398),
 ('1990s', 0.0016539338372989058),
 ('sports_x', 0.000312908445884909),
 ('pvp', 0.0025771300850820867),
 ('old_school', 0.00014017490577192103),
 ('military', 0.00015044666263495167),
 ('strategy_x', 7.81888963258284e-05),
 ('survival', 0.0011015903755644202),
 ('score_attack', 0.008490347504695824),
 ('1980s', 0.0016293627955250383),
 ('assassin', 0.0029665096554696885),
 ('violent_x', 0.001367143293886261),
 ('world_war_ii', 0.008799538597306445),
 ('war', 0.00045567902706097744),
 ('class', 0.0017621734273686662),
 ('historical', 0.0013639063259724288),
 ('co_x', 0.00020917369921269726

In [ ]:
#histograms to see what best relates to hit
# steamdf[steamdf['Hit']==1]['Metascores'].hist()
# plt.show()
# steamdf[steamdf['Hit']==0]['Metascores'].hist()
# plt.show()